In [3]:
import requests
import random
import re
import urllib

import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup


In [4]:
user_agents = ['Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/37.0.2062.94 Chrome/37.0.2062.94 Safari/537.36',
 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36',
 'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0']

In [5]:
chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path


'c:\\Users\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\chromedriver_autoinstaller\\124\\chromedriver.exe'

In [6]:
class AmazonScraper:
    base_url = "https://www.amazon.com"

    def __init__(self, url):
        
        service = webdriver.chrome.service.Service()
        options = webdriver.ChromeOptions()
        options.add_argument("--start-maximized")
        options.add_argument("--headless=new")
        options.add_argument(f"user-agent={random.choice(user_agents)}")
        options.add_experimental_option("detach", True)
        self.driver = webdriver.Chrome(service=service, options=options)

        self.search_url = url
        self.driver.get(self.search_url)
        
        
        
    def isNullElement(self, element):
        img = element.find_elements(By.XPATH, ".//img[@src]")
        if element.text.strip() or img:
            return False
        return True
    
    def find_element(self, element, locator, expression, list=True):
        result = element.find_elements(locator, expression)
        if list:
            return result
        
        return result[0] if result else None

    def getProductURLList(self):
        WebDriverWait(self.driver, 20).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 's-result-list')]")))
        
        qid = self.driver.find_element(By.XPATH, "//input[@name='qid']").get_attribute("value")
        
        
        total_page = 1

        nav = self.driver.find_element(By.XPATH, "//div[@role='navigation']")
        next_ele =  nav.find_elements(By.XPATH, "//a[contains(@class, 's-pagination-item s-pagination-next')]")
        
        if next_ele:
            total_page = int(next_ele[0].find_element(By.XPATH, "preceding::*[1]").text)

        product_urls = []
        
        curr_page = 1
        
        while True: 
            for product in self.driver.find_elements(By.XPATH, "//a[contains(@class, 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')]"):
                product_urls.append(product.get_attribute("href"))


            curr_page +=1
            
            if curr_page > total_page:
                break

            next_page_url = construct_url_params(self.search_url, {'page': curr_page, 'qid': qid, 'ref': f'sr_pg_{curr_page}'})
            self.driver.get(next_page_url)
        
        return product_urls
    
    def getLeftImage(self):
        return [img.get_attribute('src') for img in scraper.driver.find_elements(By.XPATH, "//div[@id='imageBlock']//div[@class='imgTagWrapper']//img")]

    def getProductNameAndIDFromURL(self):
        product_path = urllib.parse.urlparse(self.driver.current_url).path.strip('/').split('/')

        product_name = product_path[0].replace("-", " ")
        product_id = product_path[2]
        
        return product_name, product_id

    def parseCenterDiv(self):
        #parse centerDiv

        product_detail = {}
    #     required_div = ["featurebullets_feature_div", "bylineInfo_feature_div", "title_feature_div", "productOverview_feature_div"]
        
        centerDiv = self.driver.find_elements(By.XPATH, "//div[@id='ppd']")[0].find_element(By.XPATH, ".//div[@id='centerCol']")

        #get product title
        product_detail['product_title'] = centerDiv.find_element(By.ID, 'productTitle').text

        #get product brand
        product_detail['product_brand'] = re.sub("^Visit the|^Brand:|store$", "", centerDiv.find_element(By.ID, 'bylineInfo').text, flags=re.IGNORECASE).strip()
        product_detail['product_brand_url'] = centerDiv.find_element(By.ID, 'bylineInfo').get_attribute('href')

        #get customer reviews
        customer_reviews = centerDiv.find_elements(By.XPATH, "//div[@id='averageCustomerReviews']")
        if customer_reviews:
            product_detail['customer_reviews'] = customer_reviews[0].text.split("\n")[0]

        #get prdouct overview
#         product_detail['product_overview'] = {}
        
        product_overview_feature_div = centerDiv.find_elements(By.XPATH, "//div[@id='productOverview_feature_div']")
        if product_overview_feature_div:        
            soup = BeautifulSoup(product_overview_feature_div[0].get_attribute('innerHTML'), 'html.parser')
            for i in soup.findAll('tr'):
                td = i.findChildren('td')

                #below if elif are just for glance icons
                if td[0].find('table'):
                    td = td[0].findAll('td')[-1].findAll('span')
                elif td[0].find('img'):
                    td = td[1].findAll('span')

                product_detail[td[0].text.strip()] = td[1].text.strip()
                
        #parse about section
        #replace non ascii characters and continous spaces
        product_detail['product_about'] = ""
        product_about = centerDiv.find_elements(By.XPATH, ".//div[@id='featurebullets_feature_div']//ul")
        
        if product_about:
            product_detail['product_about'] = re.sub("\s+", " ", re.sub(r'[^\x00-\x7F]+', "", product_about[0].text))


        return product_detail

    def parseBottomDivs(self):

        productDescription = self.driver.find_elements(By.XPATH, "//div[@id='productDescription']")
        if productDescription:
            productDescription = productDescription[0].text.strip()
        product_config = {}
        misc = {}
        long_description = ""
        brand_story = ""
        detailBullets = self.driver.find_elements(By.XPATH, "//div[@id='detailBullets_feature_div' and not(@data-feature-name)]")
        if detailBullets:
            print(detailBullets[0].text)

            for li in detailBullets[0].find_elements(By.TAG_NAME, "li"):
                spans = li.find_elements(By.XPATH, ".//span/span")
                if spans:
                    product_config[spans[0].text.replace(":","").strip()] = spans[1].text.strip()
        else:
            productDetails = self.driver.find_elements(By.XPATH, "//div[@id='productDetailsNonPets_feature_div']")
            if productDetails:
                print(productDetails[0].text)


        product_information = ''
        productDetails_feature_div = self.driver.find_elements(By.XPATH, "//div[@id='productDetails_feature_div']")
        if productDetails_feature_div:
            print(productDetails_feature_div[0].text)
            product_information = productDetails_feature_div[0].text

        aplus_feature_div = self.driver.find_elements(By.XPATH, "//div[@id='aplus_feature_div' and div and normalize-space()]")

        images = []
        if aplus_feature_div:

            long_description = aplus_feature_div[0].find_element(By.XPATH, ".//div[@id='aplus']/div").text
            images = [i.get_attribute("src") for i in aplus_feature_div[0].find_elements(By.TAG_NAME, "img")]

        aplus_BS_feature_div = self.driver.find_elements(By.XPATH, "//div[@id='aplusBrandStory_feature_div' and div and normalize-space()]")

        if aplus_BS_feature_div:
            brand_story = aplus_BS_feature_div[0].find_element(By.XPATH, ".//div[@id='aplus']/div").text

            images.extend([i.get_attribute("src") for i in aplus_BS_feature_div[0].find_elements(By.TAG_NAME, "img")])


        #find all the divs where id's contains btfContent and has div which has some text or img element
        btf_contents = self.driver.find_elements(By.XPATH, "//div[contains(@id, 'btfContent') and div and normalize-space()]") 

        btf_description = ""
        for btf_content in btf_contents:
                btf_description += " " + btf_content.text.strip()
                images.extend([i.get_attribute("src") for i in btf_content.find_elements(By.TAG_NAME, "img")])


        misc = btf_description
        product_details = {
            'product_short_description': re.sub(r'[^\x00-\x7F]+', '', productDescription) if productDescription else '', 
            'product_long_description': re.sub(r'[^\x00-\x7F]+', '',long_description),
            'brand_story': re.sub(r'[^\x00-\x7F]+', '', brand_story),
            'misc': misc,
            'product_information': product_information
        }        

        product_details.update(product_config)
        return product_details, images

    def quit(self):
        self.driver.quit()

In [7]:
url = "https://www.amazon.com/Security-Cameras-Wireless-Outdoor-Detection/dp/B0CMCWYDZK/ref=sr_1_49?dib=eyJ2IjoiMSJ9.m_AiQpf2yoCwfa-K_x2q9EhrNxh-nvhecXXvGcwYrgPAvamAIy_jH_2sHXePsjpuYseIHtHkN-wU0AGWWXAqwiuA7i4y4v7aiXtKy9xNI5rlxyPIA9uGA8b4APmkG53X3Ri6w2sYAh9prpyWg4TcizKYT8bYCiGCPDfPKmLkykyp6GCo7LLEJVU8BGJDPyoTgPQeq5lEXbHDpdA12D_4V5TW7aJSEnBFSWHZCMz-v6w._jsgfcHD4Tm442QWxbZ0Re6F-rarutsvYkyjf50Rebk&dib_tag=se&keywords=smart+camera&qid=1708821756&sr=8-49"

scraper = AmazonScraper(url)

In [40]:
# scraper.driver.get(product_url)
product = {}
product_name, product_id = scraper.getProductNameAndIDFromURL()

product['id'] = product_id
product['name'] = product_name

main_image = scraper.getLeftImage()[0]
center_div = scraper.parseCenterDiv()

bottom, _ = scraper.parseBottomDivs()

product['brand'] = center_div['product_brand']

product['short_description'] = center_div['product_about']
product['long_description'] = " ".join([bottom['brand_story'], bottom['product_long_description'], bottom['product_information'], bottom['product_short_description'], bottom['misc']])


product['image_url'] = main_image

product['url'] = url

Product information
Compatible Devices iOS
Power Source Corded Electric
Connectivity Protocol Wi-Fi
Controller Type SmartThings
Mounting Type Wall Mount
Video Capture Resolution 3 MP, 2K
Color 2 Pack
Number of Items 2
Wireless Communication Technology Wi-Fi
Item Dimensions LxWxH 5 x 3 x 5 inches
Batteries Required? No
Low light technology Night Color
Alert Type Motion Only
Room Type Office, Kitchen, Living Room, Nursery
Video Capture Format MPEG-4
Unit Count 2.0 Count
Product Dimensions 5 x 3 x 5 inches
Item Weight 1.06 pounds
ASIN B0CMCWYDZK
Customer Reviews
361
4.0 out of 5 stars
Best Sellers Rank #239 in Bullet Surveillance Cameras
Date First Available November 10, 2023
Manufacturer Noahcam
Warranty & Support
Product Warranty: For warranty information about this product, please click here
Feedback
Would you like to tell us about a lower price?


In [41]:
product

{'id': 'B0CMCWYDZK',
 'name': 'Security Cameras Wireless Outdoor Detection',
 'brand': 'Noahcam',
 'short_description': 'Easy Installation - Bluetooth Qucik Connecttionlightbulb camera outdoor is easy to install and operate. Simply screw it into a light socket and open the bluetooth of your phone then follow the in-app instructions to connect to 2.4GHz/5G WiFi. No additional power supply is required, making the installation process effortless. The smart bulb camera needs to be inserted into the E27 light socket (100V~240V). The lightbulb security camera is equipped with a light socket adapter. Motion Detection & Body Detection AlarmThe wesecuu 360 wireless bulb security camera is equipped with a highly sensitive motion sensor that supports horizontal rotation of 355 degree and vertical flip of 90 degree, realizing 360 degree super wide-range accurate detection and tracking of objects. If you turn on human body detection alarm in APP, once security camera outdoor detects human body move